# Kernel Abstractions

El paquete `KernelAbstractions.jl` te permite escribir kernels similares a un kernel de GPU que se puede ejecutar en distintos dispositivos. Su intención es ser una biblioteca mínima con buen rendimiento que permita escribir código sencillo y heterogéneo. Es una alternativa a utilizar el paquete `CUDA.jl` que además permite ejecutar los mismos kernels en CPU y en GPU.

El uso de kernels para GPU proviene del hecho de que hoy en día, el uso de tarjetas gráficas para cómputo científico ha demostrado ser bastante eficiente.

## ¿Cómo programar un GPU?
En general se hace debe tener esto en cuenta:
* Modelo SPMD (_Single Process Multiple Data_)
* Se programa con una perspectiva de **camino** (llamado hilo por Nvidia)
* Un grupo de caminos se ejecutan de manera conjunta (warp/vector)
* La ejecución se organiza en bloques/grupos que pueden acceder a recursos compartidos
* La GPU se basan en el rendimiento
* Se debe tener en cuenta la arquitectura de la memoria y el uso de ancho de banda al diseñar un kernel


In [6]:
using KernelAbstractions, CUDAKernels
using CUDA
using Test

Para escribir un kernel usando `KernelAbstractions.jl` es necesario declarar una función usando el macro `@kernel`

In [12]:
@kernel function mul2(A)
    I = @index(Global)
    A[I] = 2 * A[I]
end

mul2 (generic function with 5 methods)

Para lanzar el kernel es necesario invocar la función con el **primer parámetro** siendo el **dispositivo donde ejecutar el kernel**. El **segundo argumento** es el **tamaño del grupo de trabajo**, y el **tercero** es un rango `ndrange` estático. El segundo y tercer argumento son opcionales.

Después de instanciar el kernel lo puedes lanzar llamando el kernel con los argumentos correctos y palabras clave para configurar el lanzamiento del kernel. El lanzamiento de los kernels es asíncrono, por lo que para ver los resultados es necesario _capturar el evento_ y _esperar a que termine_.

El `ndrange` se parte en bloques, cada bloque es ejecutado por **un grupo de trabajo**. Por ejemplo, funciona bien para:
* Grupo de trabajo = (4, 4)
* ndrange = (16, 16)

In [4]:
A = ones(32,32)
kernel = mul2(CPU(),16)
event = kernel(A,ndrange=size(A))
wait(event)
@show A

32×32 Matrix{Float64}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     

Para lanzar el mismo kernel con la GPU es necesario indicar `CUDADevice()` como dispositivo, así como utilizar el tipo de datos adecuado como argumento.

In [15]:
B = CUDA.ones(1024,1024)
kernel2 = mul2(CUDADevice(),16)
event = kernel2(B,ndrange=size(B))
wait(event)
@show B


Excessive output truncated after 4195339 bytes.

B = 

1024×1024 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 ⋮                        ⋮              ⋱                 ⋮              
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.

## Lenguaje de kernel

En una función que contenga la macro `@kernel` se puede utilizar el lenguaje de kernel incluíso en el paquete:

* `@Const`: Permite declarar un argumento que es constante. Es decir, no se escribe como parte del kernel ni hace referencia a otra memoria en el kernel.

* `@index`: Permite obtener el índice del entorno dentro de un kernel. Puedes obtener tanto un índice lineal como uno cartesiano.
    ### Granularidad del Índice
    * `Global`: Usado para acceder a memoria global.
    * `Group`: El índice del grupo de trabajo.
    * `Local`: El índice dentro del grupo de trabajo.
    ### Tipo de Índice
    * `Linear`: Produce un `Int64` que se usa para acceder de manera lineal a la memoria.
    * `Cartesian`: Produce un `CartesianIndex{N}` para acceder a la memoria.
    * `NTuple`: Produce un `NTuple{N}` para acceder a la memoria.
El argumento or defecto es `Linear`.

```Julia
    @index(Global, Linear)
    @index(Global, Cartesian)
    @index(Local, Cartesian)
    @index(Group, Linear)
    @index(Local, NTuple)
    @index(Global)
```

* `@localmem`: Declara una memoria local para un grupo de trabajo.

* `@private`: Declara memoria que es local para cada elemento en el grupo de trabajo. Se puede usar de manera segura para sincronizar los hilos.

* `@synchronize`: Después de esta sentencia todas las lecturas y escrituras a memoria local y global de cada hilo en el grupo de trabajo.

* `@print`: Sirve para imprimir dentro del kernel.

## Ejemplos:


In [3]:
@kernel function copy_kernel!(A, @Const B)
    I = @index(Global)
    @inbounds A[I] = B[I]
end

copy_kernel! (generic function with 5 methods)

In [4]:
function mycopy!(A::Array, B::Array)
    @assert size(A) == size(B)
    kernel = copy_kernel!(CPU(), 8)
    kernel(A,B, ndrange=length(A))
end

mycopy! (generic function with 1 method)

In [7]:
A = zeros(128,128)
B = ones(128,128)
event = mycopy!(A,B)
wait(event)
@test A == B

Test Passed
  Expression: A == B
   Evaluated: [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0] == [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

In [10]:
if has_cuda() && has_cuda_gpu()
    function mycopy!(A::CuArray, B::CuArray)
        @assert size(A) == size(B)
        copy_kernel!(CUDADevice(), 256)(A, B, ndrange=length(A))
    end

    A = CuArray{Float32}(undef, 1024)
    B = CUDA.ones(Float32, 1024)
    event = mycopy!(A,B)
    wait(event)
    @test A == B
end

Test Passed
  Expression: A == B
   Evaluated: Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] == Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

## Comparación entre CUDA.jl y KernelAbstractions.jl

En general, uno puede usar ambas librerías para construir kernels para GPU; sin embargo, los kernels construidos con `CUDA.jl` no pueden ejecutarse con el CPU, mientras que con `KernelAbstractions.jl` sí es posible. Los tipos de datos que usan ambos son los mismos (`CuArray`), a excepción de la manera de declarar memoria compartida. Para mostrar la diferencia de uso de ambos paquetes, crearemos el mismo programa usando ambas bibliotecas. El programa tiene como objetivo obtener la matriz transpuesta de una matriz, primero realizado de manera "ingenua", y luego usando memoria compartida para mejorar el uso de memoria y ancho de banda del GPU.

### Matriz Transpuesta Ingenua

In [16]:
using KernelAbstractions, CUDA, CUDAKernels

@kernel function transpose_kernel_nv!(A, @Const B)
    I,J = @index(Global, NTuple)
    @inbounds A[I,J] = B[J,I]
end

transpose_kernel_nv! (generic function with 5 methods)

In [52]:
B  = CUDA.rand(64,64)
A  = similar(B)
Ax = similar(B)

const transpose! = transpose_kernel_nv!(CUDADevice(), (32,32))
event = transpose!(A,B, ndrange=size(A))
wait(event)
@show A

64×64 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.974954   0.0790626  0.46719    …  0.22633    0.689319  0.635561
 0.0561513  0.221508   0.826807      0.170608   0.632967  0.864559
 0.752618   0.572047   0.984443      0.268353   0.414733  0.812482
 0.838736   0.870736   0.869567      0.483663   0.525199  0.904956
 0.830334   0.367194   0.584099      0.866764   0.873547  0.384195
 0.395576   0.976292   0.0638496  …  0.0383313  0.435954  0.804388
 0.310672   0.765672   0.306926      0.0805827  0.94806   0.597387
 0.756049   0.0590058  0.425904      0.819202   0.665524  0.578942
 0.679445   0.723373   0.0528807     0.188758   0.982613  0.777483
 0.593358   0.292923   0.964337      0.0864089  0.947523  0.144051
 ⋮                                ⋱                       
 0.679124   0.380334   0.383251   …  0.498911   0.890486  0.0675879
 0.739486   0.961227   0.104527      0.564903   0.348581  0.843546
 0.338623   0.275283   0.642112      0.491931   0.275179  0.422221
 0.743985   0.33210

In [19]:
@show B 

64×64 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.996059    0.91271   0.669744   …  0.590227   0.72542    0.360778
 0.802734    0.761051  0.516635      0.11815    0.796344   0.388488
 0.0012281   0.668878  0.156632      0.698013   0.576733   0.59419
 0.559936    0.351366  0.0664896     0.961207   0.494534   0.123446
 0.00266297  0.350677  0.0445517     0.705549   0.112994   0.309934
 0.352857    0.938405  0.644025   …  0.329602   0.0928931  0.0380282
 0.240857    0.803277  0.476764      0.781645   0.452563   0.049848
 0.341734    0.854166  0.0367136     0.427318   0.681104   0.800245
 0.0156344   0.151619  0.402855      0.168077   0.654153   0.64049
 0.467154    0.669903  0.731971      0.544794   0.465232   0.651381
 ⋮                                ⋱                        
 0.526856    0.95149   0.17509    …  0.788622   0.864167   0.762459
 0.210794    0.478757  0.892773      0.582176   0.343771   0.776082
 0.315545    0.97205   0.264825      0.983838   0.0130996  0.124601
 0.6094

In [49]:
function matriz_transpose_cd_nv!(A, B)
    indexX = threadIdx().x + (blockIdx().x - 1) * blockDim().x 
    indexY = threadIdx().y + (blockIdx().y - 1) * blockDim().y
    
    for j in 0:blockDim().y:blockDim().x-1
        A[indexX,indexY+j] = B[indexY + j, indexX]
    end

    return nothing
end


matriz_transpose_cd_nv! (generic function with 1 method)

In [50]:
dimGrid  = (Int(64/32),Int(64/32),1)
dimBlock = (32,8,1)

@cuda threads=dimBlock  blocks=dimGrid matriz_transpose_cd_nv!(Ax,B)

CUDA.HostKernel{typeof(matriz_transpose_cd_nv!), Tuple{CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}}}(matriz_transpose_cd_nv!, CuFunction(Ptr{Nothing} @0x00000000be4551b0, CuModule(Ptr{Nothing} @0x00000000be013aa0, CuContext(0x0000000067b3f7d0, instance 6b12de4ee8104e7d))), CUDA.KernelState(Ptr{Nothing} @0x0000000603e00000))

In [54]:
@show Ax

64×64 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                 ⋮              
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0